# **Import Modules**

In [1]:
import sys

import os

import random as pyrand

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

sys.path.insert(0, '../input/deeplearning-utils')
sys.path.insert(0, '../input/swintransformertf')

In [2]:
from utils.ops import plot

from utils.tf.ops import io, eval as tf_eval

In [3]:
from swintransformer import SwinTransformer
from swintransformer.model import Mlp

In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

import matplotlib.pyplot as plt
import matplotlib.cm as cm

import seaborn as sns

from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

from tqdm import tqdm

In [5]:
from kaggle_secrets import UserSecretsClient
from kaggle_datasets import KaggleDatasets

In [6]:
import tensorflow as tf

from tensorflow.keras import backend, optimizers, regularizers

from tensorflow.keras.layers import *

from tensorflow.keras.models import Sequential, Model

# from tensorflow.keras.applications import *

from tensorflow.keras.layers.experimental import preprocessing

import tensorflow_addons as tfa

# **Config**

In [7]:
def set_seed(tf_seed=0, np_seed=0, py_rand=0, py_hash=0):

    os.environ['PYTHONHASHSEED'] = str(py_hash)
    pyrand.seed(py_rand)

    tf.random.set_seed(tf_seed)
    np.random.seed(np_seed)

In [8]:
class Config:
    
    SEED = 2053
    
    IMAGE_SIZE = (224, 224)
    
    TRAINING = False

In [9]:
set_seed(Config.SEED)

In [10]:
try:
    
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    
    print('Running on TPU ', tpu.master())
    
except ValueError:
  
    tpu = None

if tpu is not None:
    
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)

    strategy = tf.distribute.experimental.TPUStrategy(tpu)

else:
    
    strategy = tf.distribute.get_strategy()


print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


# **Load Data**

In [11]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

ignore_order = tf.data.Options()
ignore_order.experimental_deterministic = False

In [12]:
src_dir = '../input/ci-sc22-places-and-scene-recognition/'

test_path = tf.io.gfile.glob(src_dir + '/test_images/test_images/*')

print(len(test_path))

3407


In [13]:
def read_test_example(i):
    
    image_name = os.path.split(test_path[i])[1]
    
    image = io.read_image(test_path[i])
    image = tf.expand_dims(image, axis=0)
    image = tf.image.resize(image, Config.IMAGE_SIZE)
    image = tf.cast(image, dtype=tf.float32)

    image /= 255.0
    
    return image_name, image

# **Load Model**

In [14]:
input_shape = (*Config.IMAGE_SIZE, 3)

In [15]:
with strategy.scope():
    
    backbone = SwinTransformer('swin_tiny_224', include_top=False, pretrained=True, use_tpu=True)

102039552/102036641 [==============================] - 19s 0us/step


In [16]:
def build_model():
    
    for i in range(len(backbone.layers)):
        
        if i == 2:
            continue
        
        backbone.layers[i].trainable = False
        
    for i in range(len(backbone.layers[2].layers) - 1):
        
        backbone.layers[2].layers[i].trainable = False
    
    # ===================================================================================
    
    inputs =  Input(shape=input_shape)

    features = backbone(inputs)
    
    features = Mlp(in_features=768, hidden_features=2048, out_features=1024, drop=0.6, prefix='features_0')(features)
    
    # ===================================================================================

    y_logits = Dense(6, name='y_logits')(features)
    y_proba = Activation('softmax', name='y_proba')(y_logits)
    
    # ===================================================================================

    model = Model(inputs, y_proba)
    
    # ===================================================================================
    
    optimizer = optimizers.Adam(learning_rate=1e-7, beta_1=0.9, beta_2=0.999)
    
    loss = tf.losses.CategoricalCrossentropy(label_smoothing=0.25, name='categorical_crossentropy')
    metrics = tf.metrics.CategoricalAccuracy(name='accuracy_score')
    
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    
    # ===================================================================================

    return model

In [18]:
weights_path = tf.io.gfile.glob('../input/cisc22placesandsceneswintransformerv3w/model-*.h5')

print(len(weights_path))

1


In [19]:
with strategy.scope():

    model = build_model()

model.load_weights(weights_path[0])

In [20]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
swin_tiny_224 (SwinTransform (None, 768)               27769058  
_________________________________________________________________
mlp_12 (Mlp)                 (None, 1024)              3673088   
_________________________________________________________________
y_logits (Dense)             (None, 6)                 6150      
_________________________________________________________________
y_proba (Activation)         (None, 6)                 0         
Total params: 31,448,296
Trainable params: 17,863,094
Non-trainable params: 13,585,202
_________________________________________________________________


# **Submission**

In [21]:
submission = pd.read_csv(src_dir + 'sample_submission.csv')

submission.drop([0, 1], axis=0, inplace=True)

for i in tqdm(range(len(test_path))):
    
    image_name, image = read_test_example(i)
    
    y = model.predict_on_batch(image).argmax(axis=-1)[0]
    
    submission = submission.append({'image_name': image_name, 'label': y}, ignore_index=True)

100%|██████████| 3407/3407 [01:20<00:00, 42.50it/s]


In [22]:
submission.to_csv('submission.csv', index=False, encoding='utf-8')

In [23]:
submission.head(5)

,image_name,label
0,22735.jpg,0
1,22706.jpg,0
2,13288.jpg,2
3,3863.jpg,1
4,22608.jpg,3
